# here we split the data, train the global model, generate the fisher, get the outliers and train them

In [ ]:
from results.final_result_CFAlignedFeatureTrainer_grid_search.concurrent_weights.results_plots import json_path
!poetry env use python3
!poetry env info

In [ ]:
!poetry run pip install ipykernel
!poetry run python -m ipykernel install --user --name=my_poetry_env --display-name "Python (Poetry)"

In [3]:
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"

In [ ]:
from federated_adaptive_learning_nist.data_utils import NistDataset
from federated_adaptive_learning_nist.global_trainings.global_training import GlobalTraining

gt=GlobalTraining()
nist_dataset=NistDataset()


In [ ]:
# split the writer to global and local
gt.generate_split_writers(seed=42)


In [ ]:
# Generate train and eval autoloader, and the full dataset as test loader
train_loader, eval_loader, _ = nist_dataset.build_dataset(gt.global_writers,train_rate=0.6, eval_rate=0.4, batch_size=64,seed=42)
_, _, test_loader = nist_dataset.build_dataset(gt.global_writers,train_rate=0.0, eval_rate=0.0, batch_size=64,seed=42)



### Training the global model and evaluate it. Also, generate the fisher,

In [ ]:
gt.train(trn_loader=train_loader,evl_loader=eval_loader,epochs=100,load_model=False)
gt.generate_fisher(train_loader)
gt.evaluate(tst_loader=test_loader)
gt.save_metrics_plots()

#### Generating the plot and displaying

In [ ]:
from federated_adaptive_learning_nist.utils.global_results_plot import plot_global_training_progress
from IPython.display import Image, display

json_path=PROJECT_ROOT/"results/global_results/global_metrics.json"
save_path=PROJECT_ROOT/"results/global_results/global_training_plot.png"
plot_global_training_progress(json_path=json_path,save_path=save_path)
display(Image(filename=str(save_path)))



### Generate selected writers, and then get 5-outliers


In [ ]:
gt.generate_selected_writers(seed=42)
gt.get_outliers(force_generate=False)


#### displaying the outlier selection results

In [ ]:
outlier_path=PROJECT_ROOT/"results/outliers"
display(Image(filename=str(outlier_path/"bottom_005_selected_writers.png")))
display(Image(filename=str(outlier_path/"results/all_none_contributor_writers.png")))
display(Image(filename=str(outlier_path/"results/bottom_005_selected_writers.png")))


### Training the outliers locally

In [ ]:
gt.outliers_training(batch_size=64,epochs=100)

#### ploting the outlier trainign results/plots

In [ ]:
from pathlib import Path
from IPython.display import Image, display

# root folder containing all your PNG files (adjust this)
root = PROJECT_ROOT/"results/outliers/results/plots"

# find all PNGs recursively
png_files = sorted(root.rglob("*.png"))

print(f"Found {len(png_files)} PNG files.")

# display them in Jupyter (limit if there are too many)
for path in png_files[:50]:  # adjust 50 to whatever your sanity can handle
    display(Image(filename=str(path)))
